# 🚨 Fraud Pattern Mining and JS Rule Generation (Databricks Ready)

In [ ]:

import json
import pandas as pd

def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, str(v)))
    return dict(items)

# Load fraud JSON file
with open("/dbfs/FileStore/tables/confirmed_fraud_payloads.json", "r") as f:
    data = json.load(f)

# Flatten payloads
flat_data = [flatten(txn['data']['attributes']['activity']['payload']) for txn in data]
df = pd.DataFrame(flat_data).fillna("null")
df.shape


## 🧠 Frequent Pattern Mining (FP-Growth)

In [ ]:

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

# Convert each row to a transaction of feature=value
transactions = []
for _, row in df.iterrows():
    txn = [f"{col}={val}" for col, val in row.items()]
    transactions.append(txn)

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_tf = pd.DataFrame(te_ary, columns=te.columns_)

# Run FP-Growth
frequent_itemsets = fpgrowth(df_tf, min_support=0.3, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets.head(10)


## 🛠️ Generate JavaScript Rules

In [ ]:

# Convert frequent patterns into JS rules
js_rules = []
for _, row in frequent_itemsets.iterrows():
    conditions = []
    for item in row['itemsets']:
        key, val = item.split('=')
        conditions.append(f"transaction['{key}'] === '{val}'")
    rule = "if (" + " && ".join(conditions) + ") { flagAsFraud(); }"
    js_rules.append(rule)

# Output first few rules
for rule in js_rules[:10]:
    print(rule)


In [ ]:
def execute_js_rules(transaction_json, js_rules):
    results = []

    for rule in js_rules:
        is_fraud = False
        def flagAsFraud():
            nonlocal is_fraud
            is_fraud = True

        try:
            js_func = js2py.eval_js(rule)
            js_func(transaction_json, flagAsFraud)
        except Exception as e:
            print(f"Error executing rule:\n{rule}\nReason: {e}")
            continue

        if is_fraud:
            results.append(rule)

    return results


In [ ]:
import json
from collections import defaultdict

def score_rules(log_file_path="rule_decision_log.jsonl", window=10):
    rule_stats = defaultdict(lambda: {'hits': 0, 'misses': 0})

    with open(log_file_path, 'r') as f:
        lines = f.readlines()[-window:]  # only last N
        for line in lines:
            record = json.loads(line)
            true_fraud = record.get('is_fraud', False)
            fired_rules = record.get('rules_fired', [])
            for rule in fired_rules:
                if true_fraud:
                    rule_stats[rule]['hits'] += 1
                else:
                    rule_stats[rule]['misses'] += 1

    # Score = precision: hits / (hits + misses)
    rule_scores = {}
    for rule, stats in rule_stats.items():
        total = stats['hits'] + stats['misses']
        precision = stats['hits'] / total if total else 0
        rule_scores[rule] = {"precision": round(precision, 2), "total": total}

    return rule_scores

In [ ]:
import json
from datetime import datetime

def append_to_rule_log(transaction_json, rules_fired, is_fraud, log_path="rule_decision_log.jsonl"):
    log_entry = {
        "timestamp": str(datetime.utcnow()),
        "transactionId": transaction_json.get("transactionId", "N/A"),
        "transactionAmount": transaction_json.get("transactionAmount", "N/A"),
        "is_fraud": is_fraud,
        "rules_fired": rules_fired
    }
    
    with open(log_path, "a") as f:
        f.write(json.dumps(log_entry) + "\n")
    
    print(f"✅ Logged transaction {log_entry['transactionId']} with {len(rules_fired)} rule(s) fired.")